In [1]:
!head ssh.log


-- Logs begin at Wed 2021-02-10 21:12:05 MSK, end at Fri 2021-02-19 16:40:33 MSK. --
фев 10 21:12:09 mil systemd[1]: Starting OpenBSD Secure Shell server...
фев 10 21:12:09 mil sshd[884]: Server listening on 0.0.0.0 port 22.
фев 10 21:12:09 mil sshd[884]: Server listening on :: port 22.
фев 10 21:12:09 mil systemd[1]: Started OpenBSD Secure Shell server.
фев 10 21:12:21 mil sshd[1160]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=221.131.165.86  user=root
фев 10 21:12:24 mil sshd[1160]: Failed password for root from 221.131.165.86 port 49460 ssh2
фев 10 21:12:28 mil sshd[1160]: Failed password for root from 221.131.165.86 port 49460 ssh2
фев 10 21:12:32 mil sshd[1160]: Failed password for root from 221.131.165.86 port 49460 ssh2
фев 10 21:12:33 mil sshd[1160]: Received disconnect from 221.131.165.86 port 49460:11:  [preauth]


In [2]:
def req_parser(s):
    month, day, timest, host, proc_id, other = s.split(' ',maxsplit=5)
    #print(month, day, timest, host, proc_id, sep=';') 
    #print(other)

    if 'Failed password for' in other:
        other,user,em2,ip,em,port,protocol = other.rsplit(' ',maxsplit=6)
        #print(user,ip,port,protocol, sep = ';')
        attack = 'password'

        return month, day, timest, ip, user, port, attack 


    elif 'authentication failure' in other:
        # logname= uid=0 euid=0 tty=ssh ruser= rhost=212.33.199.47  user=root
        other, user = other.rsplit(maxsplit=1)
        user = user.partition('=')[-1]
        other, ip = other.rsplit(maxsplit=1)
        ip = ip.partition('=')[-1]
        port = '00000'
        #print(user)
        #print(ip)
        attack = 'auth'


        return month, day, timest, ip, user, port, attack 

    else:

        return None

In [3]:
file_name = "ssh.log"
sshlog = open(file_name)

In [4]:
import sqlite3
conn = sqlite3.connect('ssh.db')
c = conn.cursor()

# Create table
c.execute('''DROP TABLE req ''')

c.execute('''CREATE TABLE req 
                     (
                     month text, 
                     day text, 
                     timest text, 
                     ip text,
                     user text, 
                     port integer, 
                     prot text
                     );''')

for i in sshlog:
    r = req_parser(i)
    if r and len(r) == 7:
        c.execute('INSERT INTO req VALUES (?,?,?,?,?,?,?)', r)

sshlog.close()

conn.commit()
conn.close()

ValueError: not enough values to unpack (expected 6, got 1)